In [15]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# Example view history data
data = {'UserID': [1, 1, 2, 2, 3, 3],
        'ProductID': ['A', 'B', 'A', 'C', 'B', 'C']}
df = pd.DataFrame(data)

In [16]:
df.head(10)

,UserID,ProductID
0,1,A
1,1,B
2,2,A
3,2,C
4,3,B
5,3,C


In [17]:
# Create co-occurrence matrix
product_user_matrix = pd.crosstab(df['ProductID'], df['UserID'])


In [18]:
product_user_matrix

UserID,1,2,3
ProductID,,,
A,1,1,0
B,1,0,1
C,0,1,1


In [19]:
co_occurrence = cosine_similarity(product_user_matrix)

In [20]:
co_occurrence

array([[1. , 0.5, 0.5],
       [0.5, 1. , 0.5],
       [0.5, 0.5, 1. ]])

In [21]:
# Convert to DataFrame for interpretation
co_occurrence_df = pd.DataFrame(co_occurrence, 
                                index=product_user_matrix.index,
                                columns=product_user_matrix.index)

In [22]:
co_occurrence_df

ProductID,A,B,C
ProductID,,,
A,1.0,0.5,0.5
B,0.5,1.0,0.5
C,0.5,0.5,1.0


In [23]:
# Get recommendations for product 'A'
recommendations = co_occurrence_df['A'].sort_values(ascending=False).index.tolist()
recommendations.remove('A')  # Exclude the product itself
print(recommendations)

['B', 'C']


**Python Code for User-Based Collaborative Filtering**

In [41]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# Example view history data
data = {
    'UserID': [1, 1, 2, 2, 2, 3, 3, 3, 4, 4],
    'ProductID': ['A', 'B', 'A', 'C', 'D', 'B', 'C', 'D', 'A', 'D']
}
df = pd.DataFrame(data)

# Step 1: Create User-Item Interaction Matrix
interaction_matrix = pd.crosstab(df['UserID'], df['ProductID'])
print(interaction_matrix)
# Step 2: Compute User-User Similarity
user_similarity = cosine_similarity(interaction_matrix)
user_similarity_df = pd.DataFrame(user_similarity, 
                                   index=interaction_matrix.index, 
                                   columns=interaction_matrix.index)

print(user_similarity_df[1].sort_values(ascending=False).drop(1))
# Step 3: Recommend Products
def recommend_for_user(user_id, top_n=3):
    if user_id not in user_similarity_df.index:
        return f"User {user_id} not found in the dataset."
    
    # Find similar users
    similar_users = user_similarity_df[user_id].sort_values(ascending=False).drop(user_id)
    
    # Aggregate products viewed by similar users, weighted by similarity
    recommendations = pd.Series(dtype=float)
    for similar_user, similarity_score in similar_users.items():
        # Get the products viewed by the similar user
        viewed_products = interaction_matrix.loc[similar_user]
        print(viewed_products)
        weighted_scores = viewed_products * similarity_score
        recommendations = recommendations.add(weighted_scores, fill_value=0)
    
    # Exclude products already viewed by the target user
    user_products = interaction_matrix.loc[user_id]
    recommendations = recommendations[user_products == 0]
    
    # Sort and return the top N products
    return recommendations.sort_values(ascending=False).head(top_n)

# Test the function
user_to_query = 1
recommendations = recommend_for_user(user_to_query, top_n=3)
print(f"Recommendations for user {user_to_query}:")
print(recommendations)


ProductID  A  B  C  D
UserID               
1          1  1  0  0
2          1  0  1  1
3          0  1  1  1
4          1  0  0  1
UserID
4    0.500000
2    0.408248
3    0.408248
Name: 1, dtype: float64
ProductID
A    1
B    0
C    0
D    1
Name: 4, dtype: int64
ProductID
A    1
B    0
C    1
D    1
Name: 2, dtype: int64
ProductID
A    0
B    1
C    1
D    1
Name: 3, dtype: int64
Recommendations for user 1:
ProductID
D    1.316497
C    0.816497
dtype: float64


**Python Code for Item-Based Collaborative Filtering**

In [30]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# Example view history data
data = {
    'UserID': [1, 1, 2, 2, 2, 3, 3, 3, 4, 4],
    'ProductID': ['A', 'B', 'A', 'C', 'D', 'B', 'C', 'D', 'A', 'D']
}
df = pd.DataFrame(data)

# Step 1: Create User-Item Interaction Matrix
# Each row is a user, and each column is a product. Values indicate interaction (e.g., viewed = 1)
interaction_matrix = pd.crosstab(df['UserID'], df['ProductID'])
print(interaction_matrix.T)
# Step 2: Compute Item-Item Similarity
# Using cosine similarity
item_similarity = cosine_similarity(interaction_matrix.T)
item_similarity_df = pd.DataFrame(item_similarity, 
                                   index=interaction_matrix.columns, 
                                   columns=interaction_matrix.columns)
print(item_similarity_df)

# Step 3: Recommend Similar Products
def recommend_similar_items(product_id, top_n=3):
    if product_id not in item_similarity_df.index:
        return f"Product {product_id} not in the dataset."
    
    # Sort by similarity score and exclude the product itself
    similar_items = item_similarity_df[product_id].sort_values(ascending=False)
    similar_items = similar_items.drop(product_id)  # Remove self-similarity
    return similar_items.head(top_n)

# Test the function
product_to_query = 'A'
recommendations = recommend_similar_items(product_to_query, top_n=3)
print(f"Recommendations for product {product_to_query}:")
print(recommendations)


UserID     1  2  3  4
ProductID            
A          1  1  0  1
B          1  0  1  0
C          0  1  1  0
D          0  1  1  1
ProductID         A         B         C         D
ProductID                                        
A          1.000000  0.408248  0.408248  0.666667
B          0.408248  1.000000  0.500000  0.408248
C          0.408248  0.500000  1.000000  0.816497
D          0.666667  0.408248  0.816497  1.000000
Recommendations for product A:
ProductID
D    0.666667
B    0.408248
C    0.408248
Name: A, dtype: float64
